# =====================================================================================
# <center><h1>Forecasting Spare-Part Inventory</h1></center>
# =====================================================================================

### Description

* This dataset contains vehicle service records with invoice dates, vehicle details, odometer readings, and spare part descriptions, used to analyze and predict maintenance patterns and spare parts demand.


### Import lib & models 

In [77]:
# importing necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings('ignore')

### Loading Dataset & Basic Checks

In [78]:
# loading the dataset & checking basic info
df=pd.read_csv('Spare_part_inventory.csv')
df.head()

,invoice_date,job_card_date,business_partner_name,vehicle_no,vehicle_model,current_km_reading,invoice_line_text
0,30-05-17,30-05-17,shivXXXXXXXXXX,KA03MFXXXX,BAJAJ AVENGER STREET 220,50000,ENGINE OIL
1,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,ENGINE OIL
2,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,POLISH
3,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,CONSUMABLES
4,02-06-17,31-05-17,KIRAXXXXXXXXXX,KA53ESXXXX,BAJAJ PULSAR NS 200,758,COOLANT OIL


In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28482 entries, 0 to 28481
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   invoice_date           28482 non-null  object
 1   job_card_date          28482 non-null  object
 2   business_partner_name  28482 non-null  object
 3   vehicle_no             28482 non-null  object
 4   vehicle_model          28482 non-null  object
 5   current_km_reading     28482 non-null  int64 
 6   invoice_line_text      28448 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.5+ MB


In [80]:
df.describe(include='O')

,invoice_date,job_card_date,business_partner_name,vehicle_no,vehicle_model,invoice_line_text
count,28482,28482,28482,28482,28482,28448
unique,555,553,1010,846,28,502
top,01-12-18,01-12-18,venkXXXXXXXXXX,KA53EVXXXX,BAJAJ PULSAR 150,ENGINE OIL
freq,179,179,424,1313,8633,3802


In [81]:
df["invoice_date"] = pd.to_datetime(df["invoice_date"], dayfirst=True)

df = df.sort_values(["invoice_line_text", "invoice_date"])

df.reset_index(drop=True, inplace=True)

In [82]:
daily_demand = (
    df.groupby(["invoice_line_text", "invoice_date"])
      .size()
      .reset_index(name="daily_usage")
)


In [83]:
daily_demand["avg_usage_7d"] = (
    daily_demand.groupby("invoice_line_text")["daily_usage"]
    .transform(lambda x: x.rolling(7).mean())
)

daily_demand["avg_usage_30d"] = (
    daily_demand.groupby("invoice_line_text")["daily_usage"]
    .transform(lambda x: x.rolling(30).mean())
)

daily_demand["usage_std_30d"] = (
    daily_demand.groupby("invoice_line_text")["daily_usage"]
    .transform(lambda x: x.rolling(30).std())
)

In [84]:
daily_demand["day_of_week"] = daily_demand["invoice_date"].dt.dayofweek
daily_demand["month"] = daily_demand["invoice_date"].dt.month
daily_demand["is_weekend"] = daily_demand["day_of_week"].isin([5,6]).astype(int)

In [85]:
INITIAL_STOCK = 110        # assumed starting stock
SUPPLIER_LEAD_TIME = 7   # days


In [86]:
daily_demand["simulated_stock"] = (
    INITIAL_STOCK -
    daily_demand.groupby("invoice_line_text")["daily_usage"].cumsum()
)

daily_demand["simulated_stock"] = daily_demand["simulated_stock"].clip(lower=0)


In [87]:
daily_demand["hard_stockout"] = (daily_demand["simulated_stock"] == 0).astype(int)

In [88]:
daily_demand["days_of_inventory"] = (
    daily_demand["simulated_stock"] /
    (daily_demand["avg_usage_7d"] + 1)
)


In [89]:
daily_demand["lead_time_demand"] = (
    daily_demand["avg_usage_7d"] * SUPPLIER_LEAD_TIME
)

daily_demand["stockout_next_7d"] = (
    daily_demand["simulated_stock"] <= daily_demand["lead_time_demand"]
).astype(int)


In [90]:
features = [
    "avg_usage_7d",
    "avg_usage_30d",
    "usage_std_30d",
    "days_of_inventory",
    "day_of_week",
    "month",
    "is_weekend"
]

daily_demand = daily_demand.dropna().reset_index(drop=True)


In [91]:
daily_demand.head()

,invoice_line_text,invoice_date,daily_usage,avg_usage_7d,avg_usage_30d,usage_std_30d,day_of_week,month,is_weekend,simulated_stock,hard_stockout,days_of_inventory,lead_time_demand,stockout_next_7d
0,3M OIL,2017-07-01,3,3.571429,3.100000,1.373392,5,7,1,17,0,3.718750,25.0,1
1,3M OIL,2017-07-03,2,3.000000,3.100000,1.373392,0,7,0,15,0,3.750000,21.0,1
2,3M OIL,2017-07-04,3,2.857143,3.033333,1.325697,1,7,0,12,0,3.111111,20.0,1
3,3M OIL,2017-07-06,4,2.714286,3.066667,1.337350,3,7,0,8,0,2.153846,19.0,1
4,3M OIL,2017-07-07,2,2.428571,3.066667,1.337350,4,7,0,6,0,1.750000,17.0,1


In [92]:
daily_demand['stockout_next_7d'].value_counts()

stockout_next_7d
1    6167
0    2115
Name: count, dtype: int64

In [95]:
daily_demand[daily_demand['hard_stockout'] == 1]

,invoice_line_text,invoice_date,daily_usage,avg_usage_7d,avg_usage_30d,usage_std_30d,day_of_week,month,is_weekend,simulated_stock,hard_stockout,days_of_inventory,lead_time_demand,stockout_next_7d
6,3M OIL,2017-07-10,5,3.142857,3.166667,1.366681,0,7,0,0,1,0.0,22.0,1
7,3M OIL,2017-07-11,5,3.428571,3.200000,1.399507,1,7,0,0,1,0.0,24.0,1
8,3M OIL,2017-07-12,3,3.571429,3.166667,1.391683,2,7,0,0,1,0.0,25.0,1
9,3M OIL,2017-07-13,5,3.857143,3.266667,1.412587,3,7,0,0,1,0.0,27.0,1
10,3M OIL,2017-07-14,1,3.428571,3.233333,1.454679,4,7,0,0,1,0.0,24.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8245,WHEEL RUBBER,2018-11-30,1,1.571429,1.933333,1.172481,4,11,0,0,1,0.0,11.0,1
8246,WHEEL RUBBER,2018-12-03,2,1.571429,1.966667,1.159171,0,12,0,0,1,0.0,11.0,1
8247,WHEEL RUBBER,2018-12-04,1,1.285714,1.900000,1.155198,1,12,0,0,1,0.0,9.0,1
8248,WHEEL RUBBER,2018-12-11,1,1.142857,1.900000,1.155198,1,12,0,0,1,0.0,8.0,1
